In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import mean_absolute_error


In [3]:
#只处理 notRepairedDamage为0的数据
df = pd.read_csv(r'E:/OpenData/CarPrice/used_car_train_20200313.csv',sep = ' ')
df['notRepairedDamage'].replace('-', 0.5, inplace=True)
df['notRepairedDamage'] = df['notRepairedDamage'].astype(float)
# train_data = df[df['notRepairedDamage']==0]
data_test = pd.read_csv(r'E:/OpenData/CarPrice/used_car_testA_20200313.csv',sep = ' ')
data_test['notRepairedDamage'].replace('-', 0.5, inplace=True)
data_test['notRepairedDamage'] = data_test['notRepairedDamage'].astype(float)
df.shape


(150000, 31)

In [4]:
# 预处理
def date_proc_zero(x):
    m = int(x[4:6])
    if m == 0:
        m = 1
    return x[:4] + '-' + str(m) + '-' + x[6:]


In [5]:
def parse_date(df, colname):
    newcol = colname + 'timestamp'
    df[newcol] = pd.to_datetime(df[colname].astype('str').apply(date_proc_zero))
    df[colname + '_year'] = df[newcol].dt.year
    df[colname + '_month'] = df[newcol].dt.month
    df[colname + '_day'] = df[newcol].dt.day
    df[colname + '_dayofweek'] = df[newcol].dt.dayofweek
    return df


In [6]:
train_data = df
train_data = parse_date(train_data, 'regDate')
train_data = parse_date(train_data, 'creatDate')
# 构造特征--计算车龄，以月为单位
train_data['carAge'] = (train_data['creatDate_year'] - train_data['regDate_year']) * 12 + train_data['creatDate_month'] - train_data['regDate_month']
  


In [7]:
data_test = parse_date(data_test, 'regDate')
data_test = parse_date(data_test, 'creatDate')
# 构造特征--计算车龄，以月为单位
data_test['carAge'] = (data_test['creatDate_year'] - data_test['regDate_year']) * 12 + data_test['creatDate_month'] - data_test['regDate_month']
  


In [8]:
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 42 columns):
SaleID                 150000 non-null int64
name                   150000 non-null int64
regDate                150000 non-null int64
model                  149999 non-null float64
brand                  150000 non-null int64
bodyType               145494 non-null float64
fuelType               141320 non-null float64
gearbox                144019 non-null float64
power                  150000 non-null int64
kilometer              150000 non-null float64
notRepairedDamage      150000 non-null float64
regionCode             150000 non-null int64
seller                 150000 non-null int64
offerType              150000 non-null int64
creatDate              150000 non-null int64
price                  150000 non-null int64
v_0                    150000 non-null float64
v_1                    150000 non-null float64
v_2                    150000 non-null float64
v_3             

In [9]:
#修改异常数据
train_data['power'][train_data['power']>600]=600
data_test['power'][data_test['power']>600]=600


D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
train_data = pd.get_dummies(train_data, prefix=None, prefix_sep='_', dummy_na=False, columns=['model','bodyType','gearbox','brand','fuelType','notRepairedDamage'], sparse=False, drop_first=False)


In [11]:
train_data.head()

,SaleID,name,regDate,power,kilometer,regionCode,seller,offerType,creatDate,price,...,fuelType_0.0,fuelType_1.0,fuelType_2.0,fuelType_3.0,fuelType_4.0,fuelType_5.0,fuelType_6.0,notRepairedDamage_0.0,notRepairedDamage_0.5,notRepairedDamage_1.0
0,0,736,20040402,60,12.5,1046,0,0,20160404,1850,...,1,0,0,0,0,0,0,1,0,0
1,1,2262,20030301,0,15.0,4366,0,0,20160309,3600,...,1,0,0,0,0,0,0,0,1,0
2,2,14874,20040403,163,12.5,2806,0,0,20160402,6222,...,1,0,0,0,0,0,0,1,0,0
3,3,71865,19960908,193,15.0,434,0,0,20160312,2400,...,1,0,0,0,0,0,0,1,0,0
4,4,111080,20120103,68,5.0,6977,0,0,20160313,5200,...,1,0,0,0,0,0,0,1,0,0


In [12]:
data_test = pd.get_dummies(data_test, prefix=None, prefix_sep='_', dummy_na=False, columns=['model','bodyType','gearbox','brand','fuelType','notRepairedDamage'], sparse=False, drop_first=False)


In [13]:
data_test.head()


,SaleID,name,regDate,power,kilometer,regionCode,seller,offerType,creatDate,v_0,...,fuelType_0.0,fuelType_1.0,fuelType_2.0,fuelType_3.0,fuelType_4.0,fuelType_5.0,fuelType_6.0,notRepairedDamage_0.0,notRepairedDamage_0.5,notRepairedDamage_1.0
0,150000,66932,20111212,313,15.0,1440,0,0,20160329,49.593127,...,0,1,0,0,0,0,0,1,0,0
1,150001,174960,19990211,75,12.5,5419,0,0,20160404,42.395926,...,1,0,0,0,0,0,0,0,0,1
2,150002,5356,20090304,109,7.0,5045,0,0,20160308,45.841370,...,1,0,0,0,0,0,0,1,0,0
3,150003,50688,20100405,160,7.0,4023,0,0,20160325,46.440649,...,1,0,0,0,0,0,0,1,0,0
4,150004,161428,19970703,75,15.0,3103,0,0,20160309,42.184604,...,1,0,0,0,0,0,0,1,0,0


In [14]:
missing_cols = set( train_data.columns ) - set( data_test.columns )
print(missing_cols)


{'model_247.0', 'price'}


In [15]:
data_test['model_247.0'] = 0


In [16]:
train_data.fillna(train_data.median(),inplace= True)
data_test.fillna(train_data.median(),inplace= True)


In [17]:
tags=list(train_data.columns)
print(len(tags))
print(tags)


344
['SaleID', 'name', 'regDate', 'power', 'kilometer', 'regionCode', 'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'regDatetimestamp', 'regDate_year', 'regDate_month', 'regDate_day', 'regDate_dayofweek', 'creatDatetimestamp', 'creatDate_year', 'creatDate_month', 'creatDate_day', 'creatDate_dayofweek', 'carAge', 'model_0.0', 'model_1.0', 'model_2.0', 'model_3.0', 'model_4.0', 'model_5.0', 'model_6.0', 'model_7.0', 'model_8.0', 'model_9.0', 'model_10.0', 'model_11.0', 'model_12.0', 'model_13.0', 'model_14.0', 'model_15.0', 'model_16.0', 'model_17.0', 'model_18.0', 'model_19.0', 'model_20.0', 'model_21.0', 'model_22.0', 'model_23.0', 'model_24.0', 'model_25.0', 'model_26.0', 'model_27.0', 'model_28.0', 'model_29.0', 'model_30.0', 'model_31.0', 'model_32.0', 'model_33.0', 'model_34.0', 'model_35.0', 'model_36.0', 'model_37.0', 'model_38.0', 'model_39.0', 'model_40.0', 'model_41.0',

In [18]:
tags.remove('price')
tags.remove('creatDatetimestamp')
tags.remove('SaleID')
tags.remove('regDatetimestamp')
print(len(tags))
print(tags)


340
['name', 'regDate', 'power', 'kilometer', 'regionCode', 'seller', 'offerType', 'creatDate', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'regDate_year', 'regDate_month', 'regDate_day', 'regDate_dayofweek', 'creatDate_year', 'creatDate_month', 'creatDate_day', 'creatDate_dayofweek', 'carAge', 'model_0.0', 'model_1.0', 'model_2.0', 'model_3.0', 'model_4.0', 'model_5.0', 'model_6.0', 'model_7.0', 'model_8.0', 'model_9.0', 'model_10.0', 'model_11.0', 'model_12.0', 'model_13.0', 'model_14.0', 'model_15.0', 'model_16.0', 'model_17.0', 'model_18.0', 'model_19.0', 'model_20.0', 'model_21.0', 'model_22.0', 'model_23.0', 'model_24.0', 'model_25.0', 'model_26.0', 'model_27.0', 'model_28.0', 'model_29.0', 'model_30.0', 'model_31.0', 'model_32.0', 'model_33.0', 'model_34.0', 'model_35.0', 'model_36.0', 'model_37.0', 'model_38.0', 'model_39.0', 'model_40.0', 'model_41.0', 'model_42.0', 'model_43.0', 'model_44.0', 'model_45.0', 'mod

In [19]:
#特征归一化
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(train_data[tags].values)
x = min_max_scaler.transform(train_data[tags].values)
x_ = min_max_scaler.transform(data_test[tags].values)


In [20]:
#获得y值
y = train_data['price'].values


In [21]:
#切分训练集
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.1)


In [79]:
model = keras.Sequential([
        keras.layers.Dense(400,activation='relu',input_shape=[340]), 
        keras.layers.Dense(200,activation='relu'),  
        keras.layers.Dense(200,activation='relu'),
        keras.layers.Dense(1)])
model.compile(loss='mean_absolute_error',
                optimizer='Adam')


In [89]:
model.fit(x_train,y_train,batch_size = 2048,epochs=10)   # 100+10+10+10


Epoch 1/10
135000/135000 [==============================] - 3s 23us/sample - loss: 405.8057
Epoch 2/10
135000/135000 [==============================] - 3s 21us/sample - loss: 403.2878
Epoch 3/10
135000/135000 [==============================] - 3s 21us/sample - loss: 402.2642
Epoch 4/10
135000/135000 [==============================] - 3s 22us/sample - loss: 406.8628
Epoch 5/10
135000/135000 [==============================] - 3s 21us/sample - loss: 407.3613
Epoch 6/10
135000/135000 [==============================] - 3s 20us/sample - loss: 400.3417
Epoch 7/10
135000/135000 [==============================] - 3s 20us/sample - loss: 407.2635
Epoch 8/10
135000/135000 [==============================] - 3s 20us/sample - loss: 399.3677
Epoch 9/10
135000/135000 [==============================] - 3s 20us/sample - loss: 402.9203
Epoch 10/10
135000/135000 [==============================] - 3s 23us/sample - loss: 400.5600


In [90]:
#比较训练集和验证集效果
print(mean_absolute_error(y_train,model.predict(x_train)))


392.22936861411245


In [91]:
test_pre = model.predict(x_test)
print(mean_absolute_error(y_test,test_pre))

482.09499427979785


In [64]:
#输出结果预测
y_=model.predict(x_)


In [65]:
data_test_price = pd.DataFrame(y_,columns = ['price'])
results = pd.concat([data_test['SaleID'],data_test_price],axis = 1)
results.to_csv('results0409-02.csv',sep = ',',index = None)
